We use the CIC-IDS2017 dataset to build a network intrusion detection model, then perform adversarial attacks to generate adversarial samples that evade the classification of the model. You can download the dataset from official CIC website or from Kaggle. For making adversarial attacks on intrusion detector, we use an excellent repository by Mohamed Amine Merzouk, Frederic Cuppens,Nora Boulahia-Cuppens3, and Reda Yaich. Their official github repo is [here](https://github.com/mamerzouk/adversarial_analysis/tree/master).

In [278]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import copy
import time as time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2

from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent, DeepFool, CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import PyTorchClassifier

%matplotlib inline

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [279]:
data = pd.read_csv('dataset.csv')
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [280]:
labels_name=data[' Label'].value_counts()
labels_name

 Label
BENIGN      629074
DoS         194642
DDoS        128022
PortScan     90819
Name: count, dtype: int64

In [281]:
# Transform the nominal attribute "Attack type" into binary (0 : normal / 1 : attack)
labels = (data[' Label'] != 'BENIGN').astype('int64')
labels

0          0
1          0
2          0
3          0
4          0
          ..
1042552    0
1042553    0
1042554    0
1042555    0
1042556    0
Name:  Label, Length: 1042557, dtype: int64

In [282]:
data['Class'] = labels
data.drop(' Label', inplace=True, axis=1)

In [283]:
new_labels_name=data['Class'].value_counts()
new_labels_name

Class
0    629074
1    413483
Name: count, dtype: int64

In [284]:
data.replace([-np.inf, np.inf], np.nan, inplace=True)

In [285]:
data.dropna(axis=0, inplace=True, how="any")

In [286]:
columns_to_drop = [' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets',
                  ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', 
                  'Bwd Packet Length Max', ' Bwd Packet Length Min', 'Fwd PSH Flags',' Bwd PSH Flags', 
                  ' Fwd URG Flags', ' Bwd URG Flags', ' Packet Length Mean', ' Packet Length Std',' Packet Length Variance', 
                  ' RST Flag Count', ' CWE Flag Count', ' Fwd Header Length.1','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk', 
                 ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',  'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
                  ' Subflow Fwd Bytes', 'Idle Mean', ' Idle Std', ' Idle Max',' Idle Min' ]

In [287]:
data = data.drop(columns=columns_to_drop)

In [288]:
from sklearn.model_selection import train_test_split

# Split training and test sets
df_training, df_testing = train_test_split(data, test_size=0.4, random_state=42)

# Print lengths of train and test sets
print("Train set length:", len(df_training))
print("Test set length:", len(df_testing))


Train set length: 625139
Test set length: 416760


In [289]:
l=df_training['Class'].value_counts()
l

Class
0    376952
1    248187
Name: count, dtype: int64

In [290]:
l=df_testing['Class'].value_counts()
l

Class
0    251594
1    165166
Name: count, dtype: int64

In [291]:
# Get NumPy arrays from DataFrames
nd_training = df_training.values
nd_testing = df_testing.values

In [292]:
# Separating arguments (x) from labelss (y)
x_train = nd_training[:, :-1]
y_train = nd_training[:, -1]
x_test = nd_testing[:, :-1]
y_test = nd_testing[:, -1]

# Make a copy of the data set as NumPy arrays
x_train_np = x_train.copy()
y_train_np = y_train.copy()
x_test_np = x_test.copy()
y_test_np = y_test.copy()

In [293]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train_np)

StandardScaler()

In [294]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [295]:
# Convert from numpy array to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [296]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        x = x.reshape(-1,47)
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a
        #print(y_pred.shape)
        return y_pred

In [297]:
def evaluate_predictions(predictions, real):
    ''' Evaluates the accuracy of the predictions'''
    n_correct = torch.eq(predictions, torch.argmax(real, dim=-1)).sum().item()
    accuracy = n_correct / len(predictions) * 100
    return accuracy

In [298]:
def stat_model(model, x_train, y_train, x_test, y_test):
    ''' Prints statistics about the model performances on the dataset'''
    _, predictions_train = model(x_train).max(dim=1)
    accuracy_train = evaluate_predictions(predictions=predictions_train, real=y_train)

    _, predictions_test = model(x_test).max(dim=1)
    accuracy_test = evaluate_predictions(predictions=predictions_test, real=y_test)
    
    print("Final Training Accuracy: {0:.4f}%\nFinal Testing Accuracy : {1:.4f}%"
          .format(accuracy_train, accuracy_test))
    # Move the tensors back to CPU
    label_test_final = y_test.cpu().numpy()
    predictions_test_final = predictions_test.cpu().numpy()
    

In [299]:
# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on : {}".format(device))

Training on : cuda


In [300]:
# Initialising the model
input_size=x_train.shape[1]
print('input size is ', input_size)

input size is  47


In [301]:
hidden_size=[64,64]
output_size=2
model = Network(input_size, hidden_size, output_size)
model

Network(
  (fc1): Linear(in_features=47, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [302]:
# Transfering model and data to GPU
model = model.to(device)
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [303]:
import torch.nn.functional as F

In [304]:
y_train2 = F.one_hot(y_train, 2)

In [305]:
y_train2 = y_train2.float()

In [306]:
y_test2 = F.one_hot(y_test, 2)


In [307]:
y_test2 = y_test2.float()

In [308]:
# Setting the Loss function and Adam learning rate
criterion = nn.BCEWithLogitsLoss(reduction = 'mean')
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)

In [311]:
# Variables to store the best performences (weights and accuracy)
best_model_weights = copy.deepcopy(model.state_dict())
best_accuracy = 0.0

In [246]:
# DataFrame for the learning curve plot
trace = pd.DataFrame(columns=['epoch', 'train_acc', 'test_acc'])
# Iterrating on the dataset
since = time.time()
for epoch in range(500+1):
    # Forward pass
    y_pred = model(x_train) 
    # torch.max(dim=1) returns the maximum value of each line AND its index
    _, predictions = y_pred.max(dim=1)
    # Compute accuracy
    accuracy_train = evaluate_predictions(predictions=predictions, real=y_train2)
    # Compute loss

    loss = criterion(y_pred, y_train2)

    # Testing model on the test set
    if epoch%10 == 0:
        _, predictions_test = model(x_test, softmax=True).max(dim=1)
        accuracy_test = evaluate_predictions(predictions=predictions_test, real=y_test2)
        # Keep track of the accuracies for the learning curve
        trace = pd.concat([trace, pd.DataFrame([
                    {'epoch':epoch, 'train_acc':accuracy_train, 'test_acc':accuracy_test}])], ignore_index=True)
        #trace = trace.append()
        # Save the best model's accuracy and parameters
        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_model_weights = copy.deepcopy(model.state_dict())
        # Displap statistics
        if epoch%100 == 0:
            time_elapsed = time.time() - since
            print("epoch: {0:4d} | loss: {1:.4f} | Train accuracy: {2:.4f}% | Test accuracy: {3:.4f}% [{4:.4f}%] | Running for : {5:.0f}m {6:.0f}s"
                  .format(epoch,
                          loss,
                          accuracy_train,
                          accuracy_test,
                          best_accuracy,
                          time_elapsed // 60,
                          time_elapsed % 60))

    # Zero all gradients
    optimizer.zero_grad()
    # Backward pass
    loss.backward()
    # Update weights
    optimizer.step()

# Compute the training time
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

epoch:    0 | loss: 0.6926 | Train accuracy: 41.4125% | Test accuracy: 41.5702% [41.5702%] | Running for : 0m 0s
epoch:  100 | loss: 0.0255 | Train accuracy: 99.1308% | Test accuracy: 99.0925% [99.0925%] | Running for : 0m 2s
epoch:  200 | loss: 0.0209 | Train accuracy: 99.2504% | Test accuracy: 99.2355% [99.3452%] | Running for : 0m 5s
epoch:  300 | loss: 0.0150 | Train accuracy: 99.4739% | Test accuracy: 99.4493% [99.4493%] | Running for : 0m 7s
epoch:  400 | loss: 0.0141 | Train accuracy: 99.4206% | Test accuracy: 99.3853% [99.5098%] | Running for : 0m 10s
epoch:  500 | loss: 0.0129 | Train accuracy: 99.4918% | Test accuracy: 99.4611% [99.6084%] | Running for : 0m 12s
Training complete in 0m 12s


In [247]:
torch.save(model.state_dict(), "./model_bce_loss.pytorch")

In [312]:
model.load_state_dict(torch.load("./model_bce_loss.pytorch"))

<All keys matched successfully>

In [313]:
criterion = nn.CrossEntropyLoss()

In [314]:
input_size = (1,1, 47)

# Applying the PyTorch wrapper
classifier = PyTorchClassifier(model=model, 
                               loss=criterion, 
                               optimizer=optimizer, 
                               input_shape=input_size, 
                               nb_classes=output_size)

In [315]:
from art.attacks.evasion import AutoProjectedGradientDescent

# AutoPGD

In [320]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)

In [321]:
x_test = x_test.reshape(-1,1,1,47)

In [324]:
x_test = x_test[:1000]

In [325]:
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

In [326]:
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])


Final Training Accuracy: 99.0000%
Final Testing Accuracy : 5.2000%


In [327]:
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD.csv")

(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [328]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)
x_test = x_test[:1000]
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD2.csv")

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 5.8000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


# FGSM

In [276]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.1,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM1.csv")


Final Training Accuracy: 99.0000%
Final Testing Accuracy : 27.0000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [329]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.3,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM2.csv")


Final Training Accuracy: 99.0000%
Final Testing Accuracy : 7.0000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [331]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.1, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM1.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 26.5000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [332]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.3, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM2.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 26.5000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [337]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.1,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD1.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 25.4000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [336]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.3,
                                        norm = 'inf', 
                                           eps_step=0.03,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD2.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 7.0000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [338]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW1.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 2.0000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])


In [339]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,47)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW2.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 99.0000%
Final Testing Accuracy : 2.0000%
(1000, 1, 1, 47)
torch.Size([1000, 2])
(1000, 47)
torch.Size([1000])
